In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ",")

                           
unscaled_inputs_all = raw_csv_data[:,1:-1]    
#print(raw_csv_data.head())#
targets_all = raw_csv_data[:,-1]   

2024-03-19 19:36:48.752860: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#balance the data:#
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)


In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [4]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]




In [5]:
samples_count = shuffled_inputs.shape[0]

train_samples_count =  int(0.8*samples_count)
validation_samples_count =  int(0.1*samples_count)
test__samples_count =  samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

In [6]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)



In [7]:
npz = np.load('Audiobooks_data_train.npz')

np.float = float    
np.int = int 

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)


In [8]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),                                                          
                            tf.keras.layers.Dense(output_size, activation='softmax')
                           ])                                                  

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
              
batch_size = 100
max_epochs = 100
              
    
model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          validation_data=(validation_inputs, validation_targets),
          verbose = 2
          )

Epoch 1/100
36/36 - 1s - loss: 0.5849 - accuracy: 0.6857 - val_loss: 0.5145 - val_accuracy: 0.7584 - 1s/epoch - 35ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4638 - accuracy: 0.7653 - val_loss: 0.4493 - val_accuracy: 0.7248 - 97ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4114 - accuracy: 0.7829 - val_loss: 0.4121 - val_accuracy: 0.7562 - 99ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3865 - accuracy: 0.7854 - val_loss: 0.3879 - val_accuracy: 0.8009 - 125ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3723 - accuracy: 0.7946 - val_loss: 0.3692 - val_accuracy: 0.7942 - 100ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3627 - accuracy: 0.7994 - val_loss: 0.3636 - val_accuracy: 0.8121 - 95ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3548 - accuracy: 0.8044 - val_loss: 0.3475 - val_accuracy: 0.8143 - 95ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3482 - accuracy: 0.8145 - val_loss: 0.3479 - val_accuracy: 0.8098 - 88ms/epoch - 2ms/step
Epoch 9/100
36/

In [9]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)


14/14 [==============================] - 0s 2ms/step - loss: 0.3317 - accuracy: 0.8147


In [11]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),                                                          
                            tf.keras.layers.Dense(output_size, activation='softmax')
                           ])                                                  

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
              
batch_size = 100
max_epochs = 100
 
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)    
    
model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2
          )

Epoch 1/100
36/36 - 1s - loss: 0.5866 - accuracy: 0.6655 - val_loss: 0.5161 - val_accuracy: 0.7293 - 766ms/epoch - 21ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4598 - accuracy: 0.7547 - val_loss: 0.4444 - val_accuracy: 0.7651 - 94ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4117 - accuracy: 0.7684 - val_loss: 0.4034 - val_accuracy: 0.8143 - 105ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3879 - accuracy: 0.7879 - val_loss: 0.3849 - val_accuracy: 0.8076 - 94ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3720 - accuracy: 0.7918 - val_loss: 0.3701 - val_accuracy: 0.8121 - 93ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3611 - accuracy: 0.8025 - val_loss: 0.3623 - val_accuracy: 0.8210 - 96ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3573 - accuracy: 0.8041 - val_loss: 0.3651 - val_accuracy: 0.7942 - 91ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3520 - accuracy: 0.8097 - val_loss: 0.3595 - val_accuracy: 0.8188 - 77ms/epoch - 2ms/step
Epoch 9/100
3